In [1]:
import os
os.chdir('../')

In [2]:
title_info_dict = {
    "僧坊荘園": ("AbbeyGrange", 414, 372, 331, 310),
    "花婿失踪事件": ("ACaseOfIdentity", 580, 522, 464, 435),
    "背中の曲がった男": ("CrookedMan", 373, 335, 298, 279),
    "踊る人形": ("DancingMen", 231, 207, 184, 173),
    "悪魔の足": ("DevilsFoot", 489, 440, 391, 366),
    "入院患者": ("ResidentPatient", 324, 291, 259, 243),
    "白銀号事件": ("SilverBlaze", 397, 367, 317, 297),
    "マダラのひも": ("SpeckledBand", 401, 360, 320, 300)
}

In [3]:
from utils.setup import easy_logger
from models.datasets.data_helper import MyDataHelper
import torch
import numpy as np
import h5py

SRO_ALL_INFO_FILE = "data/processed/KGCdata/All/SRO/info.hdf5"
SRO_ALL_TRAIN_FILE = "data/processed/KGCdata/All/SRO/train.hdf5"

logger = easy_logger()
data_helper = MyDataHelper(SRO_ALL_INFO_FILE, None, SRO_ALL_TRAIN_FILE, None, None, logger=logger,
                           entity_special_num=0, relation_special_num=0)

2022-12-22 19:14:03 - INFO - data_helper.py - 438 - ==========Show DataHelper==========
2022-12-22 19:14:03 - INFO - data_helper.py - 439 - ==========
2022-12-22 19:14:03 - INFO - data_helper.py - 205 - ==========Show MyRawData==========
2022-12-22 19:14:03 - INFO - data_helper.py - 206 - entity_length: 7857
2022-12-22 19:14:03 - INFO - data_helper.py - 207 - relation_length: 63
2022-12-22 19:14:03 - INFO - data_helper.py - 210 - ==========Show MyRawData==========
2022-12-22 19:14:03 - INFO - data_helper.py - 441 - ==========
2022-12-22 19:14:03 - INFO - data_helper.py - 443 - entity_special_num: 0
2022-12-22 19:14:03 - INFO - data_helper.py - 444 - relation_special_num: 0
2022-12-22 19:14:03 - INFO - data_helper.py - 445 - ==========Show DataHelper==========


In [4]:
title_info_dict

{'僧坊荘園': ('AbbeyGrange', 414, 372, 331, 310),
 '花婿失踪事件': ('ACaseOfIdentity', 580, 522, 464, 435),
 '背中の曲がった男': ('CrookedMan', 373, 335, 298, 279),
 '踊る人形': ('DancingMen', 231, 207, 184, 173),
 '悪魔の足': ('DevilsFoot', 489, 440, 391, 366),
 '入院患者': ('ResidentPatient', 324, 291, 259, 243),
 '白銀号事件': ('SilverBlaze', 397, 367, 317, 297),
 'マダラのひも': ('SpeckledBand', 401, 360, 320, 300)}

In [5]:
triple = data_helper.data.train_triple
entities = data_helper.processed_entities
relations = data_helper.processed_relations

In [6]:
from utils.hdf5 import str_list_for_hdf5

triple_str = [(entities[h], relations[r], entities[t]) for h, r, t in triple]
triple_head, triple_relation, triple_tail = zip(*triple_str)
triple_str_np = str_list_for_hdf5(triple_str)

In [13]:
import h5py
def missing_make(_title_ja, _title_en, _l100, _l, _l_name):
    i=1
    while f'{_title_en}:{_l+i}' not in triple_head:
        i+=1
        print(i)
    from_ = triple_head.index(f'{_title_en}:{_l+i}')
    to_ = len(triple_head)-triple_head[::-1].index(f'{_title_en}:{_l100}')
    print(_title_ja, from_, to_)
    path_ = SRO_ALL_TRAIN_FILE.replace('train.hdf5', f'train_{title_en}_{_l_name}.hdf5')
    with h5py.File(path_, 'w') as f:
        f.create_dataset('triple', data=np.delete(triple, slice(from_, to_), 0))
        f.create_dataset('triple_raw', data=np.delete(triple_str_np, slice(from_, to_), 0))

In [14]:
for title_ja, (title_en, l100, l090, l080, l075) in title_info_dict.items():
    missing_make(title_ja, title_en, l100, l090, 'l090')
    missing_make(title_ja, title_en, l100, l080, 'l080')
    missing_make(title_ja, title_en, l100, l075, 'l075')

僧坊荘園 1336 1476
僧坊荘園 1198 1476
僧坊荘園 1115 1476
2
花婿失踪事件 3374 3568
花婿失踪事件 3160 3568
花婿失踪事件 3075 3568
背中の曲がった男 4891 5016
背中の曲がった男 4750 5016
背中の曲がった男 4682 5016
踊る人形 5810 5885
踊る人形 5705 5885
踊る人形 5674 5885
悪魔の足 7659 7830
悪魔の足 7480 7830
悪魔の足 7355 7830
入院患者 9058 9195
入院患者 8919 9195
入院患者 8865 9195
白銀号事件 10711 10829
白銀号事件 10482 10829
白銀号事件 10401 10829
マダラのひも 12573 12755
マダラのひも 12385 12755
マダラのひも 12283 12755
